In [4]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [36]:
SEED = 42
NUM_CLASSES = 8
BATCH_SIZE = 32
EPOCHS = 50

In [32]:
if tf.test.is_gpu_available():
    print("Using GPU for training")
else:
    print("No GPU found. Using CPU instead.")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No GPU found. Using CPU instead.


## Get data

In [8]:
DATA_PATH = os.path.abspath(os.path.join('..', 'data', 'processed', 'ravdess'))

In [9]:
def load_data(data_path):
    all_data = []
    all_labels = []

    for file in os.listdir(data_path):
        if file.endswith(".npy"):
            data = np.load(os.path.join(data_path, file), allow_pickle=True)
            all_data.append(data)

            label = file.split("-")[2]
            all_labels.append(label)

    return np.array(all_data, dtype=object), np.array(all_labels)

In [10]:
all_data, all_labels = load_data(DATA_PATH)

## Preprocess data

In [11]:
def preprocess_data(data, labels):
    padded_data = pad_sequences(data, padding='post', dtype='float32')

    encoder = LabelBinarizer()
    encoded_labels = encoder.fit_transform(labels)

    X_train, X_temp, y_train, y_temp = train_test_split(padded_data, encoded_labels, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(all_data, all_labels)

In [23]:
print(X_train.shape)
print(y_train.shape)

(2012, 157, 478, 2)
(2012, 8)


## Build GRU model

In [24]:
def build_gru_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(64, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.GRU(64),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [25]:
input_shape = (X_train.shape[1], X_train.shape[2] * X_train.shape[3])
model = build_gru_model(input_shape, NUM_CLASSES)

## Train model

In [37]:
history = model.fit(
    X_train.reshape(X_train.shape[0], X_train.shape[1], -1), y_train,
    validation_data=(X_val.reshape(X_val.shape[0], X_val.shape[1], -1), y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.2281 - loss: 1.9303 - val_accuracy: 0.1995 - val_loss: 1.9539
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.2076 - loss: 1.9339 - val_accuracy: 0.2111 - val_loss: 1.9687
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.2168 - loss: 1.9211 - val_accuracy: 0.2111 - val_loss: 1.9479
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.2203 - loss: 1.9670 - val_accuracy: 0.2158 - val_loss: 1.9630
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.1954 - loss: 1.9685 - val_accuracy: 0.1972 - val_loss: 1.9505
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.1937 - loss: 1.9589 - val_accuracy: 0.1972 - val_loss: 1.9491
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.2403 - loss: 1.9229 - val_accuracy: 0.1972 - val_loss: 1.9471
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.1853 - loss: 1.9409 - val_accuracy: 0.2088 - v

KeyboardInterrupt: 

## Eval model

In [28]:
loss, accuracy = model.evaluate(X_test.reshape(X_test.shape[0], X_test.shape[1], -1), y_test, batch_size=BATCH_SIZE)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1872 - loss: 1.9333
Test Loss: 1.9093, Test Accuracy: 0.2222
